In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipyleaflet
import datetime

# Data Project: Chigago Taxi Set

## Erste Schritte
Mache dich mit der Struktur des Datensets vertraut. Öffne das Datenset <font color='orange'>bigdata/chigago/chigago.csv</font> und untersuche das Datenset mit den gelernten Methoden. Eine Kurzbeschreibung findest du unter im gleichen Ordner <font color='orange'>chigago_columns.txt</font>.

In [ ]:
df = pd.read_csv('../../src/bigdata/chicago-taxi-trips/chicago_taxi_trips_2016_01.csv')
df.head()

## Hinzufügen von Taxiunternehmen, Long und Lat
Die Informationen von Longitude und Latitude sind im Moment normalisiert. Das heißt wir müssen die Daten aus einer anderen Quelle erst verknüpfen. Wir brauchen für die weitere Auswertung nur die 'pickup' Koordinaten.

**Die Dateien liegen als .json Datei vor. Auch JSON Dateien können mit Pandas importiert werden:**

Für Longitude Latitude ist es notwendig den dtype auf float zu setzen, da sonst PANDAS denkt, es handle sich um ein Datum. Außerdem müssen wir für einen merge zwischen einem DataFrame und eine Series sicherstellen, das dieses einen Namen (dieser wird zum Spaltennamen) hat.

```python
s_beispiel = pd.read_json(<path to source>, typ='series', dtype='float')
s_beispiel.name = 'beispiel' 
```

Desweiteren wollen wir die Namen der Taxiunternehmen auf die selbe Weise importieren und verknüpfen.
<hr>

***Tipp: Alle Dateien befinden sich im Ordner Abschlussuebung***

In [ ]:
# Prüfen ob der DataFrame vorher genauso lang ist wie nacher
len(df)

In [ ]:
s_long_pick = pd.read_json('pickup_longitude.json', typ='series', dtype='float')
s_long_pick.name = 'pickup_longitude_clean'
df = pd.merge(df, s_long_pick, how='left', left_on=['pickup_longitude'], right_index=True)

In [ ]:
s_lat_pick = pd.read_json('pickup_latitude.json', typ='series', dtype='float')
s_lat_pick.name = 'pickup_latitude_clean'
df = pd.merge(df, s_lat_pick, how='left', left_on=['pickup_latitude'], right_index=True)

In [ ]:
s_company = pd.read_json('company.json', typ='series', dtype='float')
s_company.name = 'company_clean'
df = pd.merge(df, s_company, how='left', left_on=['company'], right_index=True)

In [ ]:
len(df)

## Bereinigen des DataFrames
Im Laufe der Bearbeitung werden Sie einige Fehler in den Datensets finden. Bereinigen Sie diese am besten direkt nach dem importieren. 

## Allgemeine Informationen
* Schaue dir die Werteverteilung des DataFrames mit ``.describe()`` an um einen Überblick zu erhalten. Sollten die Zahlen nur in Wissenschaftlicher Schreibweise erscheinen schreibe in eine Zeile vorher: ``pd.options.display.float_format = '{:.2f}'.format`` um die Ausgabe von Pandas zu formatieren.
* Wie viel unterschiedliche Taxifahrer gibt es in der Stadt?
* Wie viele Taxiunternehmen gibt es und wie viele Taxifahrten wurden von den verschiedenen Taxis für die Unternehmen durchgeführt

**Werteverteilung**

In [ ]:
# Finden Sie Extremwerte und filtern Sie diese ,soweit du sie schon erkennen kannst, heraus.
pd.options.display.float_format = '{:.2f}'.format
df.describe()

**Wie viele verschiedene Taxifahrer gibt es in der Stadt**

In [ ]:
len(df['taxi_id'].unique())

**Wie viele Taxiunternehmen gibt es und wie viele Taxifahrten wurden von den verschiedenen Taxis für die Unternehmen durchgeführt**

In [ ]:
df.groupby(['company_clean', 'taxi_id']).size().head()

## Ergänzen der Informationen
* Berechnen Sie die Gesamtkosten einer Fahrt und speichern Sie diese in einer Spalte mit dem Namen **trip_total**. <br>**Zusatz**: *Der Elegantere Weg nutzt die ``.sum()`` Funktion. Informieren Sie sich darüber was der ``axis=1`` der Parameter der ``.sum()`` Funktion bewirkt.*
* Stellen Sie die Verteilung der Gesamteinnahmen als Boxplot und Violinplot dar
* Berechnen Sie aus der Zeit und der Distanz die durchschnittliche Geschwindigkeit und speichern Sie diese iner einer neuen Spalte mit dem Namen **velo**
* Untersuchen Sie die Grenzwerte der neu berechneten Spalten

**Gesamtkosten**

In [ ]:
df['trip_total'] = df['fare'] + df['tips'] + df['tolls'] + df['extras']

In [ ]:
# Zusatzübung
df['trip_total'] = df[['fare', 'tips', 'tolls', 'extras']].sum(axis=1)

**Gesamteinnahmen als Boxplot und Violinplot**

In [ ]:
# Boxplot 
df.boxplot(['trip_total'], showfliers=False)

In [ ]:
# Violinplot 
fig, ax = plt.subplots(figsize=(10, 6))

# Konvertieren der Serie in listen, da pyplot keine Serien unterstützt
w = df['trip_total'].dropna().drop_duplicates().tolist()

data=[w]

ax.violinplot(data)
plt.show()

**Geschwindigkeit**

In [ ]:
df['velo'] = df['trip_miles']/(df['trip_seconds']/3600)

**Untersuchung und Bereinigung**

In [ ]:
df['velo'].describe()

In [ ]:
# Ab diesen Zeitpunkt arbeiten wir mit der Geschwindigkeit. 
# Daher müssen alle unrealistischen Werte entfernt werden.
df = df.dropna(subset=['velo'])

In [ ]:
# 9997 Dollar für einen Taxitrip sind eher unwahrscheinlich. Solche Fahrten müssen überprüft werden.
df['trip_total'].describe()

In [ ]:
# Wir setzen hier eine geschätze Grenze von 2000 für trip_total.
df = df[df['trip_total'] < 2000]

# Map Visualisation mit Leaflet

Wenn wir Datenpunkte auf einer Karte darstellen möchten, können wir die Bibliothek ipyleaflet benutzten.

## Konvertieren von Spalten in eine Liste
Zunächst müssen wir allerdings lernen, wie man die Daten aus einem DataFrame in eine Liste umwandelt. Wir  möchten jetzt eine oder mehrer Spalten in eine Liste konvertieren in der keine NaN vorkommt.Wir erreichen dies mit dem folgenden Vorgehen:

``df[['col_a', 'col_b',..].dropna().values.tolist()``

Als Ergebnis sollten wir eine verschachtelte Liste erhalten:

``[[.., .. ,..], ... ,[.., .. ,..]]``

Probiere die Spalten eine Liste zu erzeugen die nur die Werte der Spalten **'pickup_latitude_clean'**, **'pickup_longitude_clean'** enthält.

In [ ]:
lat_lon = df[['pickup_latitude_clean', 'pickup_longitude_clean']].dropna().values.tolist()
lat_lon

# iypleaflet - Daten auf Karten

Probiere dich an den folgenden beiden Beispiele und versuche diese anzuwenden:
* Heatmap: https://ipyleaflet.readthedocs.io/en/latest/api_reference/heatmap.html
* Marker Cluster: https://ipyleaflet.readthedocs.io/en/latest/api_reference/marker_cluster.html

* **Zusatz**: *Radius der Heatmap mit Schieberegeler einstellen*

**Tipps**: Das Zentrum von Chicago hat folgende Koordinaten (41.87,-87.62). Als Zoomstufe hat sich 8 als guter Startwert herausgestellt. 


**Heatmap mit Leaflet**

In [ ]:
m = ipyleaflet.Map(center=(41.87,-87.62), zoom=10)


heatmap = ipyleaflet.Heatmap(
    locations=lat_lon,
    radius=25
)

m.add_layer(heatmap);
m

**Marker Cluster** <br>
Benutze die untenstehende Funktion um die Marker für das MarkerCluster zu erstellen. Du kannst als Parameter Size angeben, also wie viele Punkte aus dem Datenset verwendet werden sollen. Taste dich **vorsichtig heran, eine ``size>500`` solltest du nicht wählen.

In [ ]:
# Transformationsfunktion
def to_marker(lat_lon, size=50):
    m = []
    
    for x in lat_lon[:size]:
        m.append(ipyleaflet.Marker(location=list(x)))
    return m

In [ ]:
m = ipyleaflet.Map(center=(41.87,-87.62), zoom=10)
    
marker_list = to_marker(lat_lon)
    
marker_cluster = ipyleaflet.MarkerCluster(
    markers=marker_list
)

m.add_layer(marker_cluster);
m

# Auswertungen

## Zeitliche Auswertungen
* Erstelle eine Spalte Tag und Uhrzeit. Zähle die Fahrten pro Tag im Monat und sortiere diese absteigend nach der Anzahl der Fahrten
* Erstelle eine Plot eines Tages. 
<br>**Tipp:** Einen Tag als Datetime erstellst du mit ``datetime.date()``. Resample außerdem die Daten auf 1h Intervalle und berechne die Anzahl der Fahrten in diesem Zeitraum.

* Erstelle eine Plot mit 3 Subplots in dem die Fahrten pro Tag, die Einnahmen pro Tag und die zurückgelegten Kilometer pro Tag analyisiert werden.

In [ ]:
df.head()

**Erstelle eine Spalte Tag und Uhrzeit**

In [ ]:
df['Day'] = pd.to_datetime(df['trip_start_timestamp'])
df['Day'] = df['Day'].dt.date
df['Day'].head()

In [ ]:
df['Time'] = pd.to_datetime(df['trip_start_timestamp'])
df['Time'] = df['Time'].dt.time
df['Time'].head()

**Fahrten pro Tag im Monat, absteigend nach der Anzahl sortiert**

In [ ]:
df.groupby('Day').size().sort_values(ascending=False)

**Erstelle eine Plot eines Tages**

In [ ]:
df_day = df[df['Day'] == datetime.date(2016, 1, 16)].copy()
df_day['trip_start_timestamp'] = pd.to_datetime(df_day['trip_start_timestamp'])
df_day.resample('1h', on='trip_start_timestamp').size().plot()

**3 Subplots in dem die Fahrten pro Tag, die Einnahmen pro Tag und die zurückgelegten Kilometer pro Tag**

In [ ]:
df.columns

In [ ]:
s_p_d = df[['Day']].groupby('Day').size()
s_p_d.name = 'trips'
s_p_d = s_p_d.to_frame()

s_tt_d = df[['Day', 'trip_total']].groupby('Day').sum()
s_km_d = df[['Day', 'trip_miles']].groupby('Day').sum()

fig, ax = plt.subplots(3,1, figsize=(15, 6))
s_p_d.plot(ax=ax[0])
s_tt_d.plot(ax=ax[1])
s_km_d.plot(ax=ax[2])

## Companys
* Lass dir die Companys und Ihrer Fahrer nach den Umsätzen der einzelnen Fahrer sortiert ausgeben
* Berechne außerdem wie profitable die Fahrer waren, also wie viel Geld diese im Schnitt pro Kilometer eingenommen haben (inkl. Trinkgeld etc.)

**Companys (aufsteigend) und Ihrer Fahrer nach den Umsätzen der einzelnen Fahrer sortiert (absteigend) ausgeben**

In [ ]:
df_company_earning = df.groupby(['company_clean', 'taxi_id'])['trip_total'].sum().reset_index()
df_company_earning.sort_values(['company_clean', 'trip_total'], ascending=[True, False]).head()

**Profitabilität der Fahrer**

In [ ]:
df_company_earning = df.groupby(['company_clean', 'taxi_id'])['trip_total', 'trip_miles'].sum().reset_index()
df_company_earning['earnings_per_miles'] = df_company_earning['trip_total']/df_company_earning['trip_miles']
df_company_earning.sort_values('earnings_per_miles', ascending=False).head()

In [ ]:
# ignorieren der Fehlerhaften inf Werte
df_company_earning[df_company_earning['trip_miles'] != 0].sort_values('earnings_per_miles', ascending=False).head()

# Weitere Möglichkeiten der Datenanalyse
Das Datenset bietet viele Möglichkeiten weitere Sachen zu erkunden. Spiele mit den Daten herum oder schaue dir neue Techniken an. Solltest du Anregungen brauchen schaue dir die Auswertung für eine Ähnliches Datenset aus New York an. Versuche einige Auswertung für Chicago durchzuführen.

* Besonders Interessant ist hier die Sektion: Anomaly Detection<br> https://www.kaggle.com/fevsea/how-much-will-it-cost-me-pre-ride-regression#Exploratory-analysis-and-anomaly-detection